<a href="https://colab.research.google.com/github/MaiNga-uit/CS2224.CH1501/blob/main/codeBook/Eval_ContentBasedImageRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation
* This step uses the groundtruth from Oxford to calculate mean average precision of relevant queries
* The precision is calculated at top 1, 4, 10 and 100 returned images. 

* Note: The model for 5k images were extracted before and upload to drive. Please refer to the following codebook for more information 
https://drive.google.com/file/d/1gl66ZCZ9zVT615hrMcoV3eaE5mweQR2i/view?usp=sharing

## Declare and create some folders

In [7]:
import os

def MakeDirWithChecked(path):
    if not os.path.isdir(path):
        os.mkdir(path)

evalDir = "/content/eval"
evalGroundTruth = "/content/eval/ground_truth"
evalQueryImg = "/content/eval/query_img"
evalRankList = "/content/eval/rank_list"

datasetDir = "/content/dataset/"
databaseDir = "/content/database"

MakeDirWithChecked(evalDir)
MakeDirWithChecked(evalGroundTruth)
MakeDirWithChecked(evalQueryImg)
MakeDirWithChecked(evalRankList)

In [15]:
import sys

from typing import List

def load_list(fname: str):
    """Plain text list loader. Reads from file separated by newlines, and returns a
    list of the file with whitespaces stripped.
    Args:
        fname (str): Name of file to be read.
    Returns:
        List[str]: A stripped list of strings, using newlines as a seperator from file.
    """

    return [e.strip() for e in open(fname, 'r').readlines()]

## Prepare query image
Prepare query image relevant to 55 queries in groundtruth and write to evalQueryImg = "/content/eval/query_img"

In [ ]:
# Don't want to crop query img?
# Just download prepared query img
# Enabled these below line and run then move directly to run classifying step

#import gdown

#modelUrl = 'https://drive.google.com/uc?id=1hUff66nwWyhBf1UG-UrcngYHPSXgcGUs' #URL cố định dùng để download.
#output = '/content/eval/query_img/query_img.zip' 
#gdown.download(modelUrl, output, quiet=False)

#!unzip -o '/content/eval/query_img/query_img.zip' -d '/content/eval'
#!rm -r '/content/eval/query_img/query_img.zip'

In [11]:
import os, sys, tarfile

def extract(tar_url, extract_path='.'):
    # print tar_url
    tar = tarfile.open(tar_url, 'r')
    for item in tar:
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])
try:

    extract(sys.argv[1] + '.tgz')
    # print 'Done.'
except:
    name = os.path.basename(sys.argv[0])
    # print name[:name.rfind('.')], '<filename>'

In [12]:
#TODO: upload groundtruth to evalGroundTruth = "/content/eval/ground_truth"
import gdown
datasetUrl = 'https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/gt_files_170407.tgz'
downloadFolder = '/content/eval/ground_truth/groundtruth.tgz' 
gdown.download(datasetUrl, downloadFolder, quiet=False)

extract('/content/eval/ground_truth/groundtruth.tgz', evalGroundTruth)
!rm '/content/eval/ground_truth/groundtruth.tgz'

In [ ]:
#TODO: dataset already upload
import gdown
!mkdir '/content/dataset/'
datasetUrl = 'https://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images.tgz'
downloadFolder = '/content/dataset/oxbuild_images.tgz' 
gdown.download(datasetUrl, downloadFolder, quiet=False)

extract('/content/dataset/oxbuild_images.tgz', '/content/dataset')
!rm '/content/dataset/oxbuild_images.tgz'

In [17]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

#query_file_name = "radcliffe_camera_1_query.txt"
def cropImage(query_file_name: str, groundtruthFolderDir: str, imgFolderDir: str, outputFolderDir: str):
  print(query_file_name)
  query_key = query_file_name.replace("_query.txt", "")
  query_detail = load_list(groundtruthFolderDir + "/" + query_file_name)
  list = query_detail[0].split()
  imgName = list[0].replace("oxc1_", "")
  img = cv2.imread(imgFolderDir + "/%s.jpg" % imgName);
  x1 = int(float(list[1]));
  y1 = int(float(list[2]));
  x2 = int(float(list[3]));
  y2 = int(float(list[4]));
  crop_img = img[y1:y2, x1:x2]

  cv2.imwrite(outputFolderDir +"/%s.jpg" % query_key, crop_img)

In [ ]:
import numpy as np
import cv2
import os
import re
from google.colab.patches import cv2_imshow

folder = evalGroundTruth
names = []
for basename in os.listdir(folder):
  name = os.path.splitext(basename)[0];
  matches = re.findall(".+_query", name)
  if len(matches) > 0:
    names.append(name)

for name in names:
  fileName = name + ".txt"
  print(fileName)
  cropImage(fileName, folder, datasetDir, evalQueryImg)

## Run classifying query by query
Browse through query_img folder and run classifying query by query. Output is relevant rank list and will be written in eval/rank_list as relevent query label

In [ ]:
# Download extracted features relevant to 5k image from Oxford
# https://drive.google.com/file/d/1gl66ZCZ9zVT615hrMcoV3eaE5mweQR2i/view?usp=sharing

In [62]:
import os

def writeToFile(dir, name, text):
  with open(dir + "/%s.txt" % name, 'w') as f:
    f.write(text)

def append_multiple_lines(file_name, lines_to_append):
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        appendEOL = False
        # Move read cursor to the start of file.
        file_object.seek(0)
        # Check if file is not empty
        data = file_object.read(100)
        if len(data) > 0:
            appendEOL = True
        # Iterate over each string in the list
        for line in lines_to_append:
            # If file is not empty then append '\n' before first line for
            # other lines always append '\n' before appending line
            if appendEOL == True:
                file_object.write("\n")
            else:
                appendEOL = True
            # Append element at the end of file
            file_object.write(line)

def writeRankResultToFile(dir, name, featureVectors):
  text = []
  for vector in featureVectors:
    nameWithExt = os.path.basename(vector[0])
    baseName = os.path.splitext(nameWithExt)[0]
    text.append(baseName)
  
  filePath = dir + "/%s.txt" % name
  append_multiple_lines(filePath, text)

In [63]:
folder = evalQueryImg
database_path = databaseDir
names = []
for query_img_path in os.listdir(folder):
  print(query_img_path)
  name = os.path.splitext(query_img_path)[0];

  query_img_categories = IU.PredictImageCategory(folder + "/" + query_img_path)
  img_features_vector = IU.CreateImageFeaturesVector(folder + "/" + query_img_path)
  print(query_img_categories)
  print(img_features_vector)
  # saving features and loading them when a query is need is slightly faster and less memory intensive than extracting the features dueing runtime
  feature_vectors = []
  for category in query_img_categories:
      category_path = os.path.join(database_path, category + '.npz')
      
      if not os.path.isfile(category_path):
          continue

      loaded_feature_vectors = np.load(category_path, allow_pickle=True)
      loaded_feature_vectors = list(loaded_feature_vectors['arr_0'])
      # only add every image once (using the image name as a reference since that shouldn't be duplicated)
      for loaded_feature_vector in loaded_feature_vectors:
          if IU.Path2Name(loaded_feature_vector[0]) in (IU.Path2Name(feature_vector[0]) for feature_vector in feature_vectors):
              continue
        
          feature_vectors.append(loaded_feature_vector)

  feature_vectors.sort(key=lambda feature_vector: distance.cosine(feature_vector[1], img_features_vector))

  writeRankResultToFile(evalRankList, name, feature_vectors)

keble_5.jpg
['church', 'palace', 'vault', 'bell_cote', 'dome']
[0.0195402  0.00923897 0.03553263 ... 0.         0.00674222 0.0054757 ]
balliol_4.jpg
['palace', 'castle', 'monastery', 'church', 'bell_cote']
[0.04792301 0.02013221 0.02333848 ... 0.00163878 0.00961255 0.00597115]
balliol_3.jpg
['palace', 'monastery', 'bell_cote', 'church', 'castle']
[0.03926379 0.02109729 0.03700565 ... 0.00330845 0.00695939 0.0057599 ]
ashmolean_5.jpg
['palace', 'library', 'triumphal_arch', 'pedestal', 'bell_cote']
[0.02441563 0.02399466 0.01679284 ... 0.0048408  0.00303912 0.00324498]
bodleian_5.jpg
['palace', 'vault', 'library', 'window_screen', 'dome']
[0.0493799  0.00490612 0.011393   ... 0.00667128 0.03389163 0.        ]
bodleian_2.jpg
['vault', 'palace', 'dome', 'church', 'library']
[0.0632548  0.00230986 0.03688461 ... 0.01219004 0.00772007 0.00045152]
magdalen_1.jpg
['church', 'bell_cote', 'monastery', 'vault', 'palace']
[0.0547462  0.01187069 0.05281353 ... 0.00029419 0.00044773 0.01134346]
radc

## Calculate mAP
Browse through all result in rank list to calculate mAP

In [64]:
def prepareValidationMaterial(grouthtruthUrl, rankListUrl, query):
  ranked_list = load_list(rankListUrl)
  pos_set = list(set(load_list(grouthtruthUrl + "/%s_good.txt" % query) + load_list(grouthtruthUrl + "/%s_ok.txt" % query)))
  junk_set = load_list(grouthtruthUrl + "/%s_junk.txt" % query)

  return ranked_list, pos_set, junk_set

In [65]:
def compute_ap(pos: List[str], amb: List[str], ranked_list: List[str]):
    """Compute average precision against a retrieved list of images. There are some bits that
    could be improved in this, but is a line-to-line port of the original C++ benchmark code.
    Args:
        pos (List[str]): List of positive samples. This is normally a conjugation of
        the good and ok samples in the ground truth data.
        amb (List[str]): List of junk samples. This is normally the junk samples in
        the ground truth data. Omitting this makes no difference in the AP.
        ranked_list (List[str]): List of retrieved images from query to be evaluated.
    Returns:
        float: Average precision against ground truth - range from 0.0 (worst) to 1.0 (best).
    """

    intersect_size, old_recall, ap = 0.0, 0.0, 0.0
    old_precision, j = 1.0, 1.0

    for e in ranked_list:
        if e in amb:
            continue

        if e in pos:
            intersect_size += 1.0

        recall = intersect_size / len(pos)
        precision = intersect_size / j
        ap += (recall - old_recall) * ((old_precision + precision) / 2.0)

        old_recall = recall
        old_precision = precision
        j += 1.0

    return ap

In [66]:
def compute_ap_at_k(pos: List[str], amb: List[str], ranked_list: List[str], k):
    """Compute average precision against a retrieved list of images. There are some bits that
    could be improved in this, but is a line-to-line port of the original C++ benchmark code.
    Args:
        pos (List[str]): List of positive samples. This is normally a conjugation of
        the good and ok samples in the ground truth data.
        amb (List[str]): List of junk samples. This is normally the junk samples in
        the ground truth data. Omitting this makes no difference in the AP.
        ranked_list (List[str]): List of retrieved images from query to be evaluated.
    Returns:
        float: Average precision against ground truth - range from 0.0 (worst) to 1.0 (best).
    """

    intersect_size, old_recall, ap = 0.0, 0.0, 0.0
    old_precision, j = 1.0, 1.0

    for e in ranked_list:
      if j <= k:
        if e in amb:
            continue

        if e in pos:
            intersect_size += 1.0

        recall = intersect_size / len(pos)
        precision = intersect_size / j
        ap += (recall - old_recall) * ((old_precision + precision) / 2.0)

        old_recall = recall
        old_precision = precision
        j += 1.0

    return ap

In [79]:
import os
import numpy as np

def cal_mAP():
  aps = []

  for query_img_path in os.listdir(evalRankList):
    if query_img_path != ".ipynb_checkpoints":
      #print(query_img_path)
      name = os.path.splitext(query_img_path)[0];
      resultRankFileDir = evalRankList + "/" + name + ".txt"
      ranked_list, pos_set, junk_set = prepareValidationMaterial(evalGroundTruth, resultRankFileDir, name)

      a = compute_ap(ranked_list, junk_set, pos_set)
      aps.append(a);
      #print(name)
      #print(a);

  return np.mean(aps)

In [80]:
import os
import numpy as np

def cal_mAP_at_k(k):
  aps = []

  for query_img_path in os.listdir(evalRankList):
    if query_img_path != ".ipynb_checkpoints":
      #print(query_img_path)
      name = os.path.splitext(query_img_path)[0];
      resultRankFileDir = evalRankList + "/" + name + ".txt"
      ranked_list, pos_set, junk_set = prepareValidationMaterial(evalGroundTruth, resultRankFileDir, name)

      a = compute_ap_at_k(ranked_list, junk_set, pos_set, k)
      aps.append(a);
      #print(name)
      #print(a);

  return np.mean(aps)

In [85]:
mAP1 = cal_mAP_at_k(1)
mAP4 = cal_mAP_at_k(4)
mAP10 = cal_mAP_at_k(10)
mAP100 = cal_mAP_at_k(100)
mAP = cal_mAP()

print("mAP@1: \t\t %s" % mAP1)
print("mAP@4: \t\t %s" % mAP4)
print("mAP@10: \t %s" % mAP10)
print("mAP@100: \t %s" % mAP100)
print("mAP: \t\t %s" % mAP)

mAP@1: 		 0.0003649085211765999
mAP@4: 		 0.0014558023458471584
mAP@10: 	 0.003504431619723861
mAP@100: 	 0.015952104736482512
mAP: 		 0.02045981028407575
